In [ ]:
import numpy as np
from collections import namedtuple
import matplotlib.pyplot as plt
import seaborn as sns
from scipy_dae.integrate import solve_dae

#### Base units
Base apparent power and frequency as defined in PowerFactory. Base torque is computed accordingly.

In [ ]:
S_base = 1e6             # [VA] base apparent power
ω_base = 2 * np.pi * 50  # [Hz] base frequency
T_base = S_base / ω_base # [Nm] base torque
print('S_base = {:g} MVA'.format(S_base * 1e-6))
print('F_base = {:g} Hz'.format(ω_base / (2 * np.pi)))
print('T_base = {:g} Nm'.format(T_base))

Load the power flow data:

In [ ]:
AC_fname = '../data/SM_with_load/adynamic_load_const_S/LD1/SM_with_load_AC.npz'
data = np.load(AC_fname, allow_pickle=True)
PF = data['PF_without_slack'].item()
PF_load = PF['loads']['LD1']
PF_gen = PF['SMs']['G1']
PF_bus = PF['buses']['Bus1']

#### Generator parameters

In [ ]:
V_base_gen = PF_gen['Vl'] * 1e3 / PF_gen['u'] # [V]
I_base_gen = PF_gen['I'] * 1e3 / PF_gen['i']  # [A]
Z_base_gen = V_base_gen / I_base_gen          # [Ω]
iGr = PF_gen['ir'] * I_base_gen               # [A]
iGi = PF_gen['ii'] * I_base_gen               # [A]
assert np.abs(PF_gen['I'] * 1e3 - np.abs(iGr + 1j * iGi)) < 1e-10

rstr, xstr = 0.2, 0.4                         # [pu] stator parameters
R_gen = rstr * Z_base_gen                     # [Ω]
X_gen = xstr * Z_base_gen                     # [Ω]
Z_gen = R_gen + 1j * X_gen                    # [Ω]

print('======= SM ======')
print('V_base = {:.2f} kV'.format(V_base_gen * 1e-3))
print('I_base = {:.2f} A'.format(I_base_gen))
print('Z_base = {:.2f} Ω'.format(Z_base_gen))
print('Z_gen = {:g} Ω'.format(Z_gen))

#### Load parameters

In [ ]:
V_base_load = PF_load['Vl'] * 1e3 / PF_load['u'] # [V]
P = PF_load['P'] * 1e6                           # [W]
Q = PF_load['Q'] * 1e6                           # [VAR]
S = P + 1j*Q                                     # [VA]
I_load = np.abs(S) / (V_base_load * np.sqrt(3))
V = (PF_load['ur'] + 1j * PF_load['ui']) * V_base_load
ϕV = np.angle(V)
ϕZ = np.angle(S)
ϕI = ϕV - ϕZ
I = I_load * np.exp(1j * ϕI)
iLr, iLi = I.real, I.imag
Z_load = V / I
assert np.abs(PF_load['ir'] * I_load - iLr) < 1e-6, 'Real part of the current does not match'
assert np.abs(PF_load['ii'] * I_load - iLi) < 1e-6, 'Imaginary part of the current does not match'
print('======= LOAD ======')
print('V_base = {:.2f} kV'.format(V_base_load * 1e-3))
print('S_load = {:g} MVA'.format((P + 1j * Q) * 1e-6))
print('I_load = {:.2f} A'.format(I_load))
print('Z_load = {:.2f} Ω'.format(Z_load))

#### Bus parameters

In [ ]:
V_base_bus = PF_bus['Vl'] * 1e3 / PF_bus['u']
uBr = PF_bus['ur'] * V_base_bus
uBi = PF_bus['ui'] * V_base_bus
assert np.abs(PF_bus['Vl'] * 1e3 - np.abs(uBr + 1j * uBi)) < 1e-10
print('======= BUS ======')
print('V_base = {:.2f} kV'.format(V_base_bus * 1e-3))

#### First algebraic equation:

In [ ]:
Z_base = V_base_bus**2 / S_base
Y_base = 1 / Z_base
print('Z_base = {:g} Ω'.format(Z_base))
print('Y_base = {:g} S'.format(Y_base))

Ggnd = 1e-4                  # couldn't figure out the units of measure
Ggnd_S = float(iGr - iLr) / uBr   # [S]
iG = iGr + 1j * iGi
iL = iLr + 1j * iLi
uB = uBr + 1j * uBi
assert np.abs(iGr - (iLr + Ggnd_S * uBr)) < 1e-10
assert np.abs(iGi - (iLi + Ggnd_S * uBi)) < 1e-10

#### Second algebraic equation:

In [ ]:
den = np.sqrt(3) * (uBr ** 2 + uBi ** 2)
assert np.abs(iLr - (P*uBr + Q*uBi) / den) < 1e-10
assert np.abs(iLi - (-Q*uBr + P*uBi) / den) < 1e-10

#### Third algebraic equation:

In [ ]:
# the voltage of the ideal generator in the synchronous machine
ve = uB + Z_gen * iG
E0, ϕG = float(np.abs(ve)), float(np.angle(ve))
assert np.abs(uBr + R_gen * iGr - X_gen * iGi - E0 * np.cos(ϕG)) < 1e-10
assert np.abs(uBi + X_gen * iGr + R_gen * iGi - E0 * np.sin(ϕG)) < 1e-10

### Torques
General parameters:

In [ ]:
n = 1     # [pu]
nref = 1 # [pu]
cosn = 1  # rated power factor

In [ ]:
ut = PF_gen['ur'] + 1j * PF_gen['ui']
it = PF_gen['ir'] + 1j * PF_gen['ii']
ψstr = (ut + rstr * it) / (1j * n)                       # [pu]
te = (it.imag * ψstr.real - it.real * ψstr.imag) / cosn  # [pu]
Te = te * T_base                                         # [Nm]
print('Stator flux: {:g} p.u.'.format(ψstr))
print('Electrical torque: {:g} p.u.'.format(te))
print('Electrical torque: {:g} Nm'.format(Te))

#### Mechanical torque

In [ ]:
dpu = 0
xmdm = 0
addmt = 0
pt = (te + dpu * n + xmdm) * n
tm = pt / n - (xmdm + dpu * n - addmt)
Tm = tm * T_base
print('Mechanical torque: {:g} p.u.'.format(tm))
print('Mechanical torque: {:g} Nm'.format(Tm))

#### Damping torque

In [ ]:
dkd, dpe = 0, 0
tdkd = dkd * (n - nref)
tdpe = dpe / n * (n - nref)
Tdkd, Tdpe = tdkd * T_base, tdpe * T_base
print('Damping torques: ({:g},{:g}) p.u.'.format(tdkd, tdpe))
print('Damping torques: ({:g},{:g}) Nm.'.format(Tdkd, Tdpe))

#### First ODE:

In [ ]:
tag = 8 # [s]
assert (tm - te - tdkd - tdpe) / tag * ω_base == 0
assert (Tm - Te - Tdkd - Tdpe) / tag * ω_base == 0

### The system of DAEs

In [ ]:
Parameters = namedtuple('Parameters', ['w_base', 'V_base', 'T_base', 'P_load', 'Q_load',
                                       'Ggnd', 'rstr', 'R_gen', 'X_gen', 'tag',
                                       'E0', 'phiG', 'cosn', 'Tm', 'I_base_gen'])

def fun(t, y, yp, params):
    w, phi, uBr, uBi, iLr, iLi, iGr, iGi = y
    wprime, phiprime = yp[:2]
    den = np.sqrt(3) * (uBr ** 2 + uBi ** 2)

    # compute the electrical torque
    ut = (uBr + 1j * uBi) / params.V_base
    it = (iGr + 1j * iGi) / params.I_base_gen
    ψstr = (ut + params.rstr * it) / 1j
    te = (it.imag * ψstr.real - it.real * ψstr.imag) / params.cosn
    Te = te * params.T_base

    F = np.zeros(8, dtype=np.common_type(y, yp))
    F[0] = wprime - (params.Tm - Te) / params.tag * params.w_base
    F[1] = phiprime
    F[2] = iGr - (iLr + params.Ggnd * uBr)
    F[3] = iGi - (iLi + params.Ggnd * uBi)
    F[4] = iLr - ( params.P_load * uBr + params.Q_load * uBi) / den
    F[5] = iLi - (-params.Q_load * uBr + params.P_load * uBi) / den
    F[6] = uBr + params.R_gen * iGr - params.X_gen * iGi - params.E0 * np.cos(params.phiG)
    F[7] = uBi + params.X_gen * iGr + params.R_gen * iGi - params.E0 * np.sin(params.phiG)
    return F

In [ ]:
params = Parameters(ω_base, V_base, T_base, P, Q, Ggnd_S, rstr, R_gen, X_gen, tag, E0, ϕG, cosn, Tm, I_base_gen)
y0 = np.array([ω_base, 0, uBr, uBi, iLr, iLi, iGr, iGi], dtype=float)
yp0 = np.zeros(8, dtype=float)
fun(0, y0, yp0, params)

In [ ]:
method = 'Radau'
atol = rtol = 1e-6
t_start, t_stop = 0, 1000
t_span = t_start, t_stop
dt = 1e-2
t_eval = np.r_[t_start : t_stop : dt]
sol = solve_dae(lambda t,y,yp: fun(t, y, yp, params), t_span, y0, yp0, atol=atol, rtol=rtol, method=method, t_eval=t_eval)

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(5,3))
ax.plot(sol.t, sol.y[0,:] / ω_base, 'k', lw=1)
ax.set_ylim([0.99, 1.01])
ax.set_xlabel('Time [s]')
ax.set_ylabel('ω [p.u.]')
sns.despine()
fig.tight_layout()